Note: this example requires data from the LifeLines-DEEP project; you have to request access from them. (https://pubmed.ncbi.nlm.nih.gov/26319774/)

In [1]:
from TeraLasso import TeraLasso
from EiGLasso import EiGLasso
from GmGM import GmGM, Dataset
from GmGM.synthetic.generate_data import threshold_dictionary

import matplotlib.pyplot as plt
import numpy as np
import timeit

from cycler import cycler
linestyle_cycler = cycler('linestyle',['-','--',':','-.'])
color_cycler = cycler('color',['k','orange','m','darkgreen'])
plt.rc('axes', prop_cycle=linestyle_cycler + color_cycler)

import igraph as ig
import pandas as pd

import muon as mu
import anndata as ad
import scanpy as sc
from anndata import AnnData
import sklearn.cluster as clust

from typing import Literal, Union

/Users/baileyandrew/mambaforge/envs/GmGM-python-demo/lib/python3.9/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/baileyandrew/mambaforge/envs/GmGM-python-demo/lib/python3.9/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/baileyandrew/mambaforge/envs/

# Setup

## Helper Functions

In [2]:
def calculate_assortativity(
    mudata: Union[mu.MuData, ad.AnnData],
) -> dict[Literal["Phylum", "Class", "Order", "Family", "Genus"], float]:
    global taxmat
    if isinstance(mudata, mu.MuData):
        precmat = mudata["metagenomics_shotgun"].varp[
            "metagenomics_shotgun-var_gmgm_connectivities"
        ].toarray()
    elif isinstance(mudata, ad.AnnData):
        precmat = mudata.varp[
            "var_gmgm_connectivities"
        ].toarray()
    else:
        raise ValueError("mudata must be of type mu.MuData or anndata.AnnData")
    g = ig.Graph.Adjacency(
        precmat != 0,
        mode="undirected",
        loops=False
    )
    return {
        tax_level: g.assortativity(taxmat[tax_level].cat.codes-1)
        for tax_level in ["Phylum", "Class", "Order", "Family", "Genus"]
    }

## Load Data

In [3]:
# Get the taxa
taxmat = pd.read_csv("../data/LL-Deep Data - Processed/ll-deep-taxmat.csv", index_col=0)
taxmat.columns = ["Domain", "Phylum", "Class", "Order", "Family", "Genus"]
taxmat = taxmat.fillna("Unknown")
taxmat = taxmat.apply(lambda x: x.astype("category"))

In [4]:
# Read in the mapping linking person ids from metabolomics and metagenomics
# (we actually constructed the latter two datasets such that the nth row matches
# up in all of them - but this still contains gender info)
map_df = pd.read_csv(
    "../data/LL-Deep Data - Processed/Map.csv",
    index_col=0
)
print(map_df.shape)

# Read in the metabolomics data
metabolomics_df = pd.read_csv(
    "../data/LL-Deep Data - Processed/Metabolomics.csv",
    index_col=0
)
print(metabolomics_df.shape)

# Read in the metagenomics data
metagenomics_df = pd.read_csv(
    "../data/LL-Deep Data - Processed/MetagenomicsShotgun.csv",
    index_col=0
)
old_shape = metagenomics_df.shape

# Only keep the species who appear in more than 20% of the people
#keep_idxs = ((counts > 0).sum(axis=0) > 0.2 * counts.shape[0]).values
#counts = counts.loc[:, keep_idxs]
#taxmat = taxmat.loc[keep_idxs, :]
keep_idxs = (metagenomics_df > 0).sum(axis=0) > 0.2 * metagenomics_df.shape[0]
metagenomics_df = metagenomics_df.loc[:, keep_idxs]
taxmat = taxmat.loc[keep_idxs.values, :]

print(old_shape, '->', metagenomics_df.shape)

(1054, 3)
(1054, 1183)
(1054, 3957) -> (1054, 564)


In [5]:
# Load our data into a MuData object
metabol_ann = AnnData(
    X = metabolomics_df.to_numpy()
)
metabol_ann.obs_names = metabolomics_df.index
metabol_ann.var_names = metabolomics_df.columns

metagen_ann = AnnData(
    X = metagenomics_df.to_numpy()
)
metagen_ann.obs_names = metabolomics_df.index # note this is the same as metabolomics
metagen_ann.var_names = metagenomics_df.columns

mudata = mu.MuData({
    "metabolomics": metabol_ann,
    "metagenomics_shotgun": metagen_ann
})

mudata.obs["Gender"] = map_df["Gender"].to_numpy()

# Log transform the data
sc.pp.log1p(mudata["metabolomics"])
sc.pp.log1p(mudata["metagenomics_shotgun"])

# GmGM

## w/o nonpara

In [6]:
GmGM(
    mudata,
    verbose=True,
    #use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    #n_comps=50,
    to_keep={
        "metabolomics-var": 1183 / 1183**2,
        "metagenomics_shotgun-var": 564 / 564**2,
        "obs": 1200 / 1054**2
    },
    threshold_method="overall",
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='metabolomics-var'
Calculating eigenvectors for axis='obs'
Calculating eigenvectors for axis='metagenomics_shotgun-var'
Calculating eigenvalues...
@0: -1707483.9041627045 (-1707489.4361432397 + 5.531980535268488 + 0) ∆inf
Converged! (@7: -1740939.4187575316)
Recomposing sparse precisions...
Converting back to MuData...


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities'

In [7]:
calculate_assortativity(mudata)

{'Phylum': 0.11738689470314732,
 'Class': 0.010345093703522575,
 'Order': -0.06284780666078045,
 'Family': 0.004783090699753132,
 'Genus': 0.0018116833315655423}

In [8]:
def to_time() -> None:
    GmGM(
        mudata,
        #use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        #n_comps=50,
        to_keep={
            "metabolomics-var": 1183 / 1183**2,
            "metagenomics_shotgun-var": 564 / 564**2,
            "obs": 1200 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

1.3620219835

## w/ nonpara

In [9]:
GmGM(
    mudata,
    verbose=True,
    use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    #n_comps=50,
    to_keep={
        "metabolomics-var": 1183 / 1183**2,
        "metagenomics_shotgun-var": 564 / 564**2,
        "obs": 1200 / 1054**2
    },
    threshold_method="overall"
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='metabolomics-var'
Calculating eigenvectors for axis='obs'
Calculating eigenvectors for axis='metagenomics_shotgun-var'
Calculating eigenvalues...
@0: -1835146.4441405768 (-1835152.1986211906 + 5.754480613893506 + 0) ∆inf
Converged! (@10: -1966807.6429533786)
Recomposing sparse precisions...
Converting back to MuData...


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities'

In [10]:
calculate_assortativity(mudata)

{'Phylum': -0.04382048233715373,
 'Class': -0.08540100047093065,
 'Order': -0.1049426423563925,
 'Family': -0.0333123983701039,
 'Genus': -0.0561269868760198}

In [11]:
def to_time() -> None:
    GmGM(
        mudata,
        use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        #n_comps=50,
        to_keep={
            "metabolomics-var": 1183 / 1183**2,
            "metagenomics_shotgun-var": 564 / 564**2,
            "obs": 1200 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

1.140237037700001

## w/o nonpara, 50pc

In [12]:
GmGM(
    mudata,
    verbose=True,
    #use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    n_comps=50,
    to_keep={
        "metabolomics-var": 1183 / 1183**2,
        "metagenomics_shotgun-var": 564 / 564**2,
        "obs": 1200 / 1054**2
    },
    threshold_method="overall",
)

Centering...
Calculating eigenvectors...
	by calculating left eigenvectors of concatenated matricizations...
Calculating eigenvalues...
@0: 15500522.380964287 (-3379.8908250230634 + 15503902.27178931 + 0) ∆inf
Converged! (@16: 11423253.864009015)
Recomposing sparse precisions...
Converting back to MuData...


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities'

In [13]:
calculate_assortativity(mudata)

{'Phylum': 0.2491494112614761,
 'Class': 0.20346105595238942,
 'Order': 0.06773398118119817,
 'Family': -0.002136928943167747,
 'Genus': 0.059627669114881626}

In [14]:
def to_time() -> None:
    GmGM(
        mudata,
        #use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        n_comps=50,
        to_keep={
            "metabolomics-var": 1183 / 1183**2,
            "metagenomics_shotgun-var": 564 / 564**2,
            "obs": 1200 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

1.2235617457000003

## w/ nonpara, 50pc

In [15]:
GmGM(
    mudata,
    verbose=True,
    use_nonparanormal_skeptic=True,
    nonparanormal_evec_backend="COCA",
    n_comps=50,
    to_keep={
        "metabolomics-var": 1183 / 1183**2,
        "metagenomics_shotgun-var": 564 / 564**2,
        "obs": 1200 / 1054**2
    },
    threshold_method="overall",
)

Centering...
Calculating eigenvectors...
	by calculating left eigenvectors of concatenated matricizations...
Calculating eigenvalues...
@0: 2300580.35889995 (-3457.187310962975 + 2304037.546210913 + 0) ∆inf
Converged! (@25: 1190802.8371083408)
Recomposing sparse precisions...
Converting back to MuData...


MuData object with n_obs × n_vars = 1054 × 1747
  obs:	'Gender'
  uns:	'obs_neighbors_gmgm'
  obsp:	'obs_gmgm_connectivities'
  2 modalities
    metabolomics:	1054 x 1183
      uns:	'log1p', 'metabolomics-var_neighbors_gmgm'
      varp:	'metabolomics-var_gmgm_connectivities'
    metagenomics_shotgun:	1054 x 564
      uns:	'log1p', 'metagenomics_shotgun-var_neighbors_gmgm'
      varp:	'metagenomics_shotgun-var_gmgm_connectivities'

In [16]:
calculate_assortativity(mudata)

{'Phylum': 0.20324365252641244,
 'Class': 0.1434982314941994,
 'Order': -0.048375184220409526,
 'Family': -0.05809242961312629,
 'Genus': 0.01814890164484217}

In [17]:
def to_time() -> None:
    GmGM(
        mudata,
        use_nonparanormal_skeptic=True,
        nonparanormal_evec_backend="COCA",
        n_comps=50,
        to_keep={
            "metabolomics-var": 1183 / 1183**2,
            "metagenomics_shotgun-var": 564 / 564**2,
            "obs": 1200 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

8.5592378417

## w/o nonpara w/o metabol

In [18]:
GmGM(
    mudata["metagenomics_shotgun"],
    verbose=True,
    #use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    #n_comps=50,
    to_keep={
        "var": 564 / 564**2,
        "obs": 1200 / 1054**2
    },
    threshold_method="overall",
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='var'
Calculating eigenvectors for axis='obs'
Calculating eigenvalues...
@0: -566088.0161710007 (-566094.2859140185 + 6.269743017843572 + 0) ∆inf
Converged! (@10: -589782.8896450654)
Recomposing sparse precisions...
Converting back to AnnData...


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [19]:
calculate_assortativity(mudata["metagenomics_shotgun"])

{'Phylum': -0.021649610719968463,
 'Class': -0.06197070129673916,
 'Order': -0.18903785084844535,
 'Family': -0.01615142857896596,
 'Genus': -0.03081067929473613}

In [20]:
def to_time() -> None:
    GmGM(
        mudata["metagenomics_shotgun"],
        #use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        #n_comps=50,
        to_keep={
            "var": 564 / 564**2,
            "obs": 1200 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

0.46486409999999695

## w/ nonpara, w/o metabol

In [21]:
GmGM(
    mudata["metagenomics_shotgun"],
    verbose=True,
    use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    #n_comps=50,
    to_keep={
        "var": 564 / 564**2,
        "obs": 1200 / 1054**2
    },
    threshold_method="overall",
)

Centering...
Calculating eigenvectors...
	by calculating gram matrices and then eigendecomposing...
Calculating eigenvectors for axis='var'
Calculating eigenvectors for axis='obs'
Calculating eigenvalues...
@0: -566118.0533103225 (-566124.245307187 + 6.1919968645482335 + 0) ∆inf
Converged! (@11: -590597.1354492181)
Recomposing sparse precisions...
Converting back to AnnData...


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [22]:
calculate_assortativity(mudata["metagenomics_shotgun"])

{'Phylum': -0.04382048233715373,
 'Class': -0.08540100047093065,
 'Order': -0.1049426423563925,
 'Family': -0.0333123983701039,
 'Genus': -0.0561269868760198}

In [23]:
def to_time() -> None:
    GmGM(
        mudata["metagenomics_shotgun"],
        use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        #n_comps=50,
        to_keep={
            "var": 564 / 564**2,
            "obs": 1200 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

0.7764470248999971

## w/o nonpara, w/o metabol, 50pc

In [24]:
GmGM(
    mudata["metagenomics_shotgun"],
    verbose=True,
    #use_nonparanormal_skeptic=True,
    #nonparanormal_evec_backend="COCA",
    n_comps=50,
    to_keep={
        "var": 564 / 564**2,
        "obs": 1200 / 1054**2
    },
    threshold_method="overall",
)

Centering...
Calculating eigenvectors...
	by calculating SVD...
Calculating eigenvalues...
@0: 13945988.771733945 (-1686.3123899860582 + 13947675.084123932 + 0) ∆inf
Converged! (@16: 8755437.389918238)
Recomposing sparse precisions...
Converting back to AnnData...


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [25]:
calculate_assortativity(mudata["metagenomics_shotgun"])

{'Phylum': 0.24212871092707128,
 'Class': 0.1925347981044774,
 'Order': 0.06571586481621407,
 'Family': 0.001145140044832476,
 'Genus': 0.0764822913233742}

In [26]:
def to_time() -> None:
    GmGM(
        mudata["metagenomics_shotgun"],
        #use_nonparanormal_skeptic=True,
        #nonparanormal_evec_backend="COCA",
        n_comps=50,
        to_keep={
            "var": 564 / 564**2,
            "obs": 1200 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

0.4137910085000073

## w/ nonpara, w/o metabol, 50pc

In [27]:
GmGM(
    mudata["metagenomics_shotgun"],
    verbose=True,
    use_nonparanormal_skeptic=True,
    nonparanormal_evec_backend="COCA",
    n_comps=50,
    to_keep={
        "var": 564 / 564**2,
        "obs": 1200 / 1054**2
    },
    threshold_method="overall",
)

Centering...
Calculating eigenvectors...
	by calculating left eigenvectors of concatenated matricizations...
Calculating eigenvalues...
@0: 539021.091890617 (-1718.0168997342569 + 540739.1087903513 + 0) ∆inf
Converged! (@19: 342571.11713876884)
Recomposing sparse precisions...
Converting back to AnnData...


AnnData object with n_obs × n_vars = 1054 × 564
    uns: 'log1p', 'metagenomics_shotgun-var_neighbors_gmgm', 'obs_neighbors_gmgm', 'var_neighbors_gmgm'
    obsp: 'obs_gmgm_connectivities'
    varp: 'metagenomics_shotgun-var_gmgm_connectivities', 'var_gmgm_connectivities'

In [28]:
calculate_assortativity(mudata["metagenomics_shotgun"])

{'Phylum': 0.19886174464247328,
 'Class': 0.16110645991138073,
 'Order': -0.04315302272994758,
 'Family': -0.06013727631131085,
 'Genus': 0.026297132241856334}

In [29]:
def to_time() -> None:
    GmGM(
        mudata["metagenomics_shotgun"],
        use_nonparanormal_skeptic=True,
        nonparanormal_evec_backend="COCA",
        n_comps=50,
        to_keep={
            "var": 564 / 564**2,
            "obs": 1200 / 1054**2
        },
        threshold_method="overall"
    )

np.mean(timeit.repeat(to_time, number=1, repeat=10))

2.3153864750000026

# TeraLasso

## w/o nonpara w/o metabol

In [39]:
ds = Dataset.from_AnnData(mudata["metagenomics_shotgun"])
TeraLasso(
    ds,
    10
    #use_nonparanormal_skeptic=True,
)
test = ds.to_AnnData()
test

[[10.0,10.0]]


In [ ]:
test.varp["var_gmgm_connectivities"]

<564x564 sparse array of type '<class 'numpy.float64'>'
	with 318096 stored elements in Compressed Sparse Row format>

In [ ]:
(test.varp["var_gmgm_connectivities"].tocoo().data > 1e-8).sum()

318096

In [ ]:
test.varp["var_gmgm_connectivities"].tocoo().data

array([138548.01071465, 182518.80248436, 193744.07657008, ...,
        27464.46177753,   7132.0111473 ,  27173.272989  ])